## Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os

from tqdm.notebook import tqdm
import pandas as pd
from llama_index.core.response.notebook_utils import display_source_node

In [4]:
import nest_asyncio 

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

## Arguments input

In [6]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Review Rec Bot - Yelp Review Rec Bot",
    RUN_NAME="025_modify_prompt_with_tree_summarize",
    RUN_DESCRIPTION="""
# Objective
Make the response from the LLM more adapted to the task of recommending places.

# Implementation
- ResponseMode=TreeSummarize
- Customize prompt

## Changelog
    """,
    TESTING=False,
    DEBUG= False,
    OBSERVABILITY=True,
    LOG_TO_MLFLOW=True,
    CREATE_RETRIEVAL_EVAL_DATASET=False,
    RECREATE_RESPONSE_EVAL_DATASET=False,
    # Need to put to RECREATE_INDEX after the other RECREATE is to be able to access the information from those variables and modify them 
    RECREATE_INDEX=False,
)
ARGS

{
  "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
  "RUN_NAME": "025_modify_prompt_with_tree_summarize",
  "RUN_DESCRIPTION": "\n# Objective\nMake the response from the LLM more adapted to the task of recommending places.\n\n# Implementation\n- ResponseMode=TreeSummarize\n- Customize prompt\n\n## Changelog\n    ",
  "TESTING": false,
  "DEBUG": false,
  "OBSERVABILITY": true,
  "LOG_TO_MLFLOW": true,
  "CREATE_RETRIEVAL_EVAL_DATASET": false,
  "RECREATE_RESPONSE_EVAL_DATASET": false,
  "RECREATE_INDEX": false
}

## Load config

In [7]:
from src.run.cfg import RunConfig

In [8]:
cfg = RunConfig()
cfg.init(ARGS)

2024-09-21 17:20:36.130 | INFO     | src.run.cfg:init:111 - Starting Observability server with Phoenix...
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/nampq/.phoenix
2024-09-21 17:20:40.982 | INFO     | src.run.cfg:init:132 - Setting up MLflow experiment Review Rec Bot - Yelp Review Rec Bot - run 025_modify_prompt_with_tree_summarize...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


2024-09-21 17:20:41.221 | INFO     | src.run.cfg:init:141 - Notebook-generated artifacts are persisted at data/025_modify_prompt_with_tree_summarize


In [9]:
cfg

{
  "args": {
    "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
    "RUN_NAME": "025_modify_prompt_with_tree_summarize",
    "RUN_DESCRIPTION": "\n# Objective\nMake the response from the LLM more adapted to the task of recommending places.\n\n# Implementation\n- ResponseMode=TreeSummarize\n- Customize prompt\n\n## Changelog\n    ",
    "TESTING": false,
    "DEBUG": false,
    "OBSERVABILITY": true,
    "LOG_TO_MLFLOW": true,
    "CREATE_RETRIEVAL_EVAL_DATASET": false,
    "RECREATE_RESPONSE_EVAL_DATASET": false,
    "RECREATE_INDEX": false
  },
  "app_name": "review_rec_bot",
  "storage_context_persist_dp": "/home/nampq/Desktop/projects/personal-bot/data/031_rerun/storage_context",
  "vector_db": "qdrant",
  "db_collection": "review_rec_bot_031_rerun",
  "db_collection_fp": "/home/nampq/Desktop/projects/personal-bot/data/031_rerun/chroma_db",
  "notebook_cache_dp": "data/025_modify_prompt_with_tree_summarize",
  "data_fp": "../data/yelp_dataset/sample/sample_100_biz/denom

## Set up logger to collect additional info

In [10]:
collect_fp = f"{cfg.notebook_cache_dp}/collect.log"
logger.add(collect_fp, filter=lambda record: "[COLLECT]" in record["message"], mode='w')

1

## Load input data

In [11]:
data = pd.read_parquet(cfg.data_fp)
data = data.assign(
    biz_categories=lambda df: df['biz_categories'].str.split(', '),
    date=lambda df: df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
)
logger.info(f"[COLLECT] {len(data)}")
data.iloc[0]

2024-09-21 17:20:41.407 | INFO     | __main__:<module>:6 - [COLLECT] 5240


review_id                                      L0jv8c2FbpWSlfNC6bbUEA
user_id                                        bFPdtzu11Oi0f92EAcjqmg
business_id                                    IDtLPgUrqorrpqSLdfMhZQ
review_stars                                                        5
useful                                                              0
funny                                                               0
cool                                                                0
text                What a great addition to the Funk Zone!  Grab ...
date                                              2016-10-13T22:50:47
biz_name                                         Helena Avenue Bakery
biz_address                                     131 Anacapa St, Ste C
biz_city                                                Santa Barbara
biz_state                                                          CA
biz_postal_code                                                 93101
biz_latitude        

In [12]:
input_data = data
if ARGS.TESTING: 
    input_data = data[:20]
logger.info(f"[COLLECT] {len(input_data)=}")

2024-09-21 17:20:42.042 | INFO     | __main__:<module>:4 - [COLLECT] len(input_data)=5240


In [13]:
input_data.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'text', 'date', 'biz_name', 'biz_address', 'biz_city',
       'biz_state', 'biz_postal_code', 'biz_latitude', 'biz_longitude',
       'biz_stars', 'biz_review_count', 'biz_is_open', 'biz_attributes',
       'biz_categories', 'biz_hours'],
      dtype='object')

## Prepare documents

In [14]:
from llama_index.core import Document

documents = []
embedding_visible_metadata = ["review_stars", "biz_name", "biz_address", "biz_city", "biz_state", "biz_categories"]
excluded_embed_metadata_keys = [k for k in input_data.columns if k not in embedding_visible_metadata]

for i, row in tqdm(input_data.iterrows(), total=len(input_data)):
    record = row.to_dict()
    text = record['text']
    metadata = {k: v for k, v in record.items() if k not in ('text')}

    doc = Document(
        text=text,
        metadata=metadata,
        excluded_embed_metadata_keys=excluded_embed_metadata_keys,
        excluded_llm_metadata_keys=excluded_embed_metadata_keys
    )
    documents.append(doc)
logger.info(f"[COLLECT] {len(documents)=}")

  0%|          | 0/5240 [00:00<?, ?it/s]

2024-09-21 17:20:44.623 | INFO     | __main__:<module>:19 - [COLLECT] len(documents)=5240


### check document embedding text

In [15]:
from llama_index.core.schema import MetadataMode

In [16]:
document = documents[0]
print(document.get_content(metadata_mode=MetadataMode.EMBED))

review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

What a great addition to the Funk Zone!  Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.


## Set up LLM

In [17]:
llm, embed_model = cfg.setup_llm()

In [18]:
print(cfg.llm_cfg.model_dump_json(indent=2))

{
  "llm_provider": "gemini",
  "llm_model_name": "models/gemini-1.5-flash",
  "embedding_provider": "huggingface",
  "embedding_model_name": "/home/nampq/Desktop/projects/personal-bot/data/Alibaba-NLP/gte-multilingual-base",
  "embedding_model_dim": 768,
  "ollama__host": "localhost",
  "ollama__port": 11434
}


In [19]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = llm

## Vector Store

In [20]:
import qdrant_client
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

from src.run.orchestrator import RunOrchestrator

In [21]:
qdrantdb = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

aqdrantdb = qdrant_client.AsyncQdrantClient(
    host="localhost",
    port=6333
)

RunOrchestrator.setup_db(cfg=cfg, qdrantdb=qdrantdb)

db_collection = qdrantdb.get_collection(
    cfg.db_collection
)
vector_store = QdrantVectorStore(
    client=qdrantdb,
    collection_name=cfg.db_collection,
    aclient=aqdrantdb,
    enable_hybrid=False,
    prefer_grpc=True
)

WARNI [llama_index.vector_stores.qdrant.base] Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


## Index Embeddings

In [22]:
import time
import pickle
from multiprocessing import set_start_method
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SemanticSplitterNodeParser

In [23]:
chunker = SemanticSplitterNodeParser
chunker_cfg = {
    "buffer_size": 1,
    "breakpoint_percentile_threshold": 95,
    "embed_model": embed_model
}

In [24]:
documents = documents[:10]

In [25]:
t0 = time.perf_counter()
db_collection_count = db_collection.points_count

if db_collection_count > 0 and ARGS.RECREATE_INDEX == False:
    logger.info(f"Loading Storage Context from {cfg.storage_context_persist_dp}")
    docstore = SimpleDocumentStore.from_persist_dir(persist_dir=cfg.storage_context_persist_dp)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    nodes = list(docstore.docs.values())
else:
    logger.info(f"Creating new DB index...")
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache

    pipeline = IngestionPipeline(
        transformations=[
            chunker(**chunker_cfg),
            embed_model,
        ],
        vector_store=vector_store
    )
    num_workers = None
    logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers, show_progress=True)

    docstore = SimpleDocumentStore()
    await docstore.async_add_documents(nodes)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    logger.info(f"Persisting Storage Context to {cfg.storage_context_persist_dp}...")
    storage_context.persist(cfg.storage_context_persist_dp)

t1 = time.perf_counter()

2024-09-21 17:21:00.323 | INFO     | __main__:<module>:5 - Loading Storage Context from /home/nampq/Desktop/projects/personal-bot/data/031_rerun/storage_context


In [26]:
logger.info(f"Indexing {len(documents)} documents into VectorStoreIndex took {t1 - t0:,.0f}s")
logger.info(f"[COLLECT] {len(nodes)=}")

2024-09-21 17:21:00.512 | INFO     | __main__:<module>:1 - Indexing 10 documents into VectorStoreIndex took 0s
2024-09-21 17:21:00.513 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=20


## Analyze Chunks

In [27]:
for i, node in enumerate(nodes[:5]):
    print(f"\n\n===========Node {i+1}============")
    print(node.metadata)
    print(node.get_text())



===========Node 1============
{'review_id': 'L0jv8c2FbpWSlfNC6bbUEA', 'user_id': 'bFPdtzu11Oi0f92EAcjqmg', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-10-13T22:50:47', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly'

## Construct Retriever

In [28]:
from llama_index.core.retrievers import VectorIndexRetriever

In [29]:
print(cfg.retrieval_cfg.model_dump_json(indent=2))

{
  "retrieval_top_k": 50,
  "retrieval_dense_top_k": 50,
  "retrieval_sparse_top_k": 50,
  "retrieval_simiarity_cutoff": null,
  "rerank_top_k": 10,
  "rerank_model_name": "BAAI/bge-reranker-v2-m3"
}


In [30]:
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
vector_retriever = VectorIndexRetriever(
    index=index,
    vector_store_query_mode='mmr',
    similarity_top_k=cfg.retrieval_cfg.retrieval_dense_top_k,
)

In [31]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=5,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)
bm25_retrieval_results = bm25_retriever.retrieve("Nashville, grocery")
for result in bm25_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

DEBUG [bm25s] Building index from IDs objects


**Node ID:** 553a4a2e-22ea-4b5b-a8d0-82a6b09079e3<br>**Similarity:** 0.0<br>**Text:** I love how creative everything was.
I CANT STOP DAYDREAMING ABOUT THE FOOD. Hands down, best brunch experience of my life. I don't even like french toast usually and theres was by far the most delicious food thats ever entered my mouth. I am literally drooling while writing this.
The chicken and waffles were unreal and standout A++++. 
I love the "flinstones pushup-pop" drink so creative and clever.
Even the deviled eggs were the best I've personally ever had. They do everything right even when it comes to the music. It was super fun and such a great experience! Ignore any review thats not 4 stars because they must be delusional.<br>**Metadata:** {'review_id': 'z_fgvINjKZCw5RgByaTxxw', 'user_id': 'dldfg-X_QbBkhR2DOsQFWg', 'business_id': 'QdN72BWoyFypdGJhhI5r7g', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-11-10T16:52:33', 'biz_name': 'Bar One', 'biz_address': '767 S 9th St', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19147', 'biz_latitude': 39.9398245705, 'biz_longitude': -75.1574465632, 'biz_stars': 4.0, 'biz_review_count': 65, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'full_bar'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': None, 'divey': False, 'intimate': False, 'trendy': None, 'upscale': False, 'classy': None, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': None, 'street': True, 'validated': None, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'False', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': True, 'breakfast': False}", 'HappyHour': 'True', 'HasTV': 'True', 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': None, 'WiFi': "u'free'"}, 'biz_categories': ['Cocktail Bars', 'Bars', 'Italian', 'Nightlife', 'Restaurants'], 'biz_hours': {'Friday': '12:0-2:0', 'Monday': '16:0-0:0', 'Saturday': '11:0-2:0', 'Sunday': '11:0-0:0', 'Thursday': '12:0-0:0', 'Tuesday': '16:0-0:0', 'Wednesday': '16:0-0:0'}}<br>

**Node ID:** 7d27d2e6-a3db-4a84-9acb-b3803bb682ae<br>**Similarity:** 0.0<br>**Text:** Some unique flavors too. Not much else to say but an amazing ice cream store that others should try to emulate.<br>**Metadata:** {'review_id': '6odPSjoPWOyeDQpgJbdGaA', 'user_id': 'jLr7vcDg43TqS2OK4ROvPQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-03-30T23:22:54', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

**Node ID:** 003f3cdf-6934-46c0-be90-c466ecf7b5e8<br>**Similarity:** 0.0<br>**Text:** Super tasty and enormous portions.<br>**Metadata:** {'review_id': '6odPSjoPWOyeDQpgJbdGaA', 'user_id': 'jLr7vcDg43TqS2OK4ROvPQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-03-30T23:22:54', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

**Node ID:** 0e0d54d1-4329-486d-a4d9-66e04873893b<br>**Similarity:** 0.0<br>**Text:** Possibly the best breakfast sandwich EVER.  On a biscuit!<br>**Metadata:** {'review_id': 'LFpaQzYkP5Pzm5lEjJpTRw', 'user_id': 'Ttb21DpWqphQMNSS5TXgMA', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-01-15T00:36:04', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'True', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries'], 'biz_hours': {'Friday': '8:0-14:0', 'Monday': '0:0-0:0', 'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0', 'Thursday': '8:0-14:0', 'Tuesday': '8:0-14:0', 'Wednesday': '8:0-14:0'}}<br>

**Node ID:** a8248e27-4017-4258-80f3-788eb11ed4f8<br>**Similarity:** 0.0<br>**Text:** Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.<br>**Metadata:** {'review_id': 'L0jv8c2FbpWSlfNC6bbUEA', 'user_id': 'bFPdtzu11Oi0f92EAcjqmg', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-10-13T22:50:47', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'True', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries'], 'biz_hours': {'Friday': '8:0-14:0', 'Monday': '0:0-0:0', 'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0', 'Thursday': '8:0-14:0', 'Tuesday': '8:0-14:0', 'Wednesday': '8:0-14:0'}}<br>

In [32]:
test_query = "Received confirmation of my order at 5 pm"
vector_results = vector_retriever.retrieve(test_query)
bm25_results = bm25_retriever.retrieve(test_query)

found = False
for vector_result in vector_results[:10]:
    target_id = bm25_results[0].id_
    if vector_result.id_ == target_id:
        found = True
        break
# assert found, "can not find the original node in top 10 vector results"
# assert bm25_results[0].id_ == vector_results[0].id_, "different vector_retriever nodes vs bm25"

In [33]:
from llama_index.core.retrievers import QueryFusionRetriever

query_gen_prompt = """
You are helpful assistant that expands and input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs sentences.
You should apply different techniques to create new strings. Here are sine example techniques:
- Technique 1 - Optimize for full-text search: Repharse the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output: "cold brew coffee hanoi
- Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question.

Generate at least {num_queries} new strings by interating over the technique in order. For example, your first generated string should always use technique 1, technique 2. If ru of techniques then re-iterate from start.

Only return the strings. Never include the chosen technique.

Input query: {query}\n
New strings:\n
"""
from llama_index.llms.gemini import Gemini
llm = Gemini(model="models/gemini-1.5-flash")

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    llm=llm,
    similarity_top_k=10,
    num_queries=2,
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt
)

## Test retrieval

In [34]:
query = "What are some places to enjoy cold brew coffee"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.LLM))
    print("\n")

Generated queries:
cold brew coffee
I love trying out new cold brew coffee spots! My favorite place is [Place Name] - they have the best cold brew in town. It's so smooth and flavorful, and they have a great selection of different roasts. I highly recommend checking it out!


**Node ID:** 99395c10-efa2-41df-8b74-8f2ec8b023e0<br>**Similarity:** 0.06530936012691697<br>**Text:** So much to like about this place--the beautiful Mercer Building location, the spaciousness, the lovely, friendly baristas. But there are a couple of important cons for me. Biggest being, the music is constant and too loud, ruining everything you go to a coffee shop to do: read, work, reflect, or enjoy a good conversation. When I asked for it to be turned down, they told me they can't--it just plays at one level all the time, and the speakers  are everywhere so you can't get away from it. It sucks.<br>

review_stars: 3
biz_name: Roast Coffeehouse and Wine Bar
biz_address: 10359 104 Street NW
biz_city: Edmonton
biz_state: AB
biz_categories: ['Coffee & Tea', 'Food', 'Cafes', 'Bars', 'Wine Bars', 'Restaurants', 'Nightlife']

So much to like about this place--the beautiful Mercer Building location, the spaciousness, the lovely, friendly baristas. But there are a couple of important cons for me. Biggest being, the music is constant and too loud, ruining everything you go to a coffee shop to do: read, work, reflect, or enjoy a good conversation. When I asked for it to be turned down, they told me they can't--it just plays at one level all the time, and the speakers  are everywhere so you can't get away from it. It sucks.




**Node ID:** 8c4df29f-641e-46c3-a9d8-8bee00d8c655<br>**Similarity:** 0.06218347862183478<br>**Text:** Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good.<br>

review_stars: 4
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good.




...

review_stars: 4
biz_name: Romano's Macaroni Grill
biz_address: 5505 S Virginia St
biz_city: Reno
biz_state: NV
biz_categories: ['Restaurants', 'Italian']

Staff are as friendly as it gets.




**Node ID:** d2c37891-35d6-43f6-9752-cb024b84a49e<br>**Similarity:** 0.03333333333333333<br>**Text:** Secondly, if you're a tea drinker, you're out of luck. They don't serve the tea in pots, but huge, sloppy individual filters--it makes your beverage totally ungainly, hard to drink and is really off putting.

Pros: gluten free goodies and yummy Italian sodas. I'd love to spend more time here, but the music usually drives me out after 10 minutes.<br>

review_stars: 3
biz_name: Roast Coffeehouse and Wine Bar
biz_address: 10359 104 Street NW
biz_city: Edmonton
biz_state: AB
biz_categories: ['Coffee & Tea', 'Food', 'Cafes', 'Bars', 'Wine Bars', 'Restaurants', 'Nightlife']

Secondly, if you're a tea drinker, you're out of luck. They don't serve the tea in pots, but huge, sloppy individual filters--it makes your beverage totally ungainly, hard to drink and is really off putting.

Pros: gluten free goodies and yummy Italian sodas. I'd love to spend more time here, but the music usually drives me out after 10 minutes.




**Node ID:** 3514a9a8-b4fe-4194-b053-e0ab21719e2e<br>**Similarity:** 0.03200204813108039<br>**Text:** Also no judgement for eating all the brownie sundae alone :) lol<br>

review_stars: 5
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food']

Also no judgement for eating all the brownie sundae alone :) lol




## Retrieval Evaluation

### Synthetic

In [35]:
from src.run.eval.retrieval import RetrievalEvaluator

In [36]:
retrieval_evaluator = RetrievalEvaluator()
retrieval_evaluator.generate_synthetic_dataset(cfg, nodes)

2024-09-21 09:51:30.449 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:34 - Creating new retrieval eval dataset at data/025_modify_prompt_with_tree_summarize/retrieval_synthetic_eval_dataset.json...
2024-09-21 09:51:30.450 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:38 - Sampling 20 nodes for retrieval evaluation...
2024-09-21 09:51:31.554 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:65 - Creating new synthetic retrieval dataset...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:07<00:00,  1.80it/s]
2024-09-21 09:51:38.789 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:73 - Loading retrieval_eval_nodes from data/025_modify_prompt_with_tree_summarize/retrieval_synthetic_eval_dataset.json...


In [37]:
retrieval_evaluator.retrieval_eval_dataset.queries

{'25246855-574a-4dcc-b727-34975cdd2dc4': 'What are some other great places to visit in the Funk Zone?',
 'ea6d6b25-3ceb-4465-b7b8-b7a8cb75c160': 'What are some businesses known for having friendly staff?',
 '62565394-e147-4210-8bcb-698c1a8aadf9': 'What are some other tap rooms in Indianapolis that have a rotating selection of beers?',
 'e75c3057-394b-49d6-83a7-9d21610dea8b': 'What are some good places to get affordable dinner and lunch to-go?',
 'dccff7cc-f232-4447-b2e7-c09846f66144': 'What are some ice cream shops known for their delicious and unique flavors, friendly service, and generous sampling policy?',
 'b70527f8-e839-48c3-afd1-1c0b45099090': 'What are some other places that offer tastings and good food in Santa Barbara?',
 'b1b9d99d-eaee-4acf-8b1c-cf72bf8e70a0': 'What are some highly-rated brunch restaurants known for their creative and delicious food?',
 'd77319f0-10cc-47f7-926a-5809f20af712': 'What are some restaurants that offer excellent customer service and a good dining e

In [43]:
retrieval_eval_results_df, retrieval_eval_results_full_df = await retrieval_evaluator.aevaluate(cfg, retriever)

Generated queries:
Funk Zone great places visit
The Funk Zone is a great place to visit in Santa Barbara. There are tons of art galleries, tasting rooms, and restaurants to explore. I highly recommend checking out the Santa Barbara Public Market for some delicious food and drinks. The Funk Zone is also home to some great breweries, so be sure to check those out as well.
Generated queries:
friendly staff businesses
The staff at [Business Name] are always so friendly and helpful. They really make you feel welcome and go above and beyond to make sure you have a great experience. I highly recommend this place!
Generated queries:
tap rooms indianapolis rotating beers
I love going to tap rooms in Indianapolis that have a rotating selection of beers. It's always exciting to try something new! I recently went to [Tap Room Name] and they had a great selection of local brews. I highly recommend checking it out if you're looking for a new place to try.
Generated queries:
affordable dinner lunch t

In [44]:
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_50_retrieval_eval,0.846154,0.846154,0.084615,0.846154,0.846154,0.186231


In [45]:
retrieval_eval_results_full_df

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
0,What are some other great places to visit in t...,[37be3c44-96ee-4a76-b054-00461b69a63e],"[37be3c44-96ee-4a76-b054-00461b69a63e, c1e0511...",1.0,1.0,0.1,1.0,1.0,0.220092
1,What are some businesses known for having frie...,[716413ae-6eb9-4ff6-8737-959217e4ccbc],"[716413ae-6eb9-4ff6-8737-959217e4ccbc, 2c67c79...",1.0,1.0,0.1,1.0,1.0,0.220092
2,What are some other tap rooms in Indianapolis ...,[2c67c79e-0733-405a-b0f4-a4a3630a6047],"[2c67c79e-0733-405a-b0f4-a4a3630a6047, 7d27d2e...",1.0,1.0,0.1,1.0,1.0,0.220092
3,What are some good places to get affordable di...,[9fbd5ea5-ada6-4888-a4e8-3ca29b2beab8],"[9fbd5ea5-ada6-4888-a4e8-3ca29b2beab8, 82d3c38...",1.0,1.0,0.1,1.0,1.0,0.220092
4,What are some ice cream shops known for their ...,[6330dafa-19f0-45c9-9810-9297fef2a162],"[6330dafa-19f0-45c9-9810-9297fef2a162, 7d27d2e...",1.0,1.0,0.1,1.0,1.0,0.220092
5,What are some other places that offer tastings...,[a8248e27-4017-4258-80f3-788eb11ed4f8],"[a8248e27-4017-4258-80f3-788eb11ed4f8, 8c4df29...",1.0,1.0,0.1,1.0,1.0,0.220092
6,What are some highly-rated brunch restaurants ...,[553a4a2e-22ea-4b5b-a8d0-82a6b09079e3],"[553a4a2e-22ea-4b5b-a8d0-82a6b09079e3, cc38402...",1.0,1.0,0.1,1.0,1.0,0.220092
7,What are some restaurants that offer excellent...,[c1e05113-d367-4f2b-8910-c163b144c2c9],"[cc384023-0468-4b06-94f2-903bdd98fb11, 8c4df29...",0.0,0.0,0.0,0.0,0.0,0.000000
8,What are some restaurants with excellent servi...,[cc384023-0468-4b06-94f2-903bdd98fb11],"[cc384023-0468-4b06-94f2-903bdd98fb11, 8c4df29...",1.0,1.0,0.1,1.0,1.0,0.220092
9,What are some restaurants known for their larg...,[003f3cdf-6934-46c0-be90-c466ecf7b5e8],"[003f3cdf-6934-46c0-be90-c466ecf7b5e8, 0e0d54d...",1.0,1.0,0.1,1.0,1.0,0.220092


## Error Analysis

In [46]:
retrieval_eval_irrelevance_df = (
    retrieval_eval_results_full_df
    .loc[lambda df: df['hit_rate'].lt(1)]
    .sort_values(['hit_rate', 'mrr', 'precision', 'recall', 'ap', 'ndcg'])
)
with pd.option_context('display.max_colwidth', 400):
    display(retrieval_eval_irrelevance_df)

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
7,What are some restaurants that offer excellent customer service and a good dining experience?,[c1e05113-d367-4f2b-8910-c163b144c2c9],"[cc384023-0468-4b06-94f2-903bdd98fb11, 8c4df29f-641e-46c3-a9d8-8bee00d8c655, 553a4a2e-22ea-4b5b-a8d0-82a6b09079e3, 0e0d54d1-4329-486d-a4d9-66e04873893b, 6330dafa-19f0-45c9-9810-9297fef2a162, 37be3c44-96ee-4a76-b054-00461b69a63e, a8248e27-4017-4258-80f3-788eb11ed4f8, 7d27d2e6-a3db-4a84-9acb-b3803bb682ae, 909e5e89-e168-4427-b745-291de49a670a, ba65ed5f-0b6a-4072-ac5a-4297acdeff15]",0.0,0.0,0.0,0.0,0.0,0.0
10,What are some places that serve delicious food?,[0daba217-4083-4d8e-ba00-da8497310fb9],"[cc384023-0468-4b06-94f2-903bdd98fb11, 553a4a2e-22ea-4b5b-a8d0-82a6b09079e3, 0e0d54d1-4329-486d-a4d9-66e04873893b, e6db66a7-38a6-4b6d-9630-627a20647cfb, 8c4df29f-641e-46c3-a9d8-8bee00d8c655, a8248e27-4017-4258-80f3-788eb11ed4f8, 6330dafa-19f0-45c9-9810-9297fef2a162, 7d27d2e6-a3db-4a84-9acb-b3803bb682ae, ba65ed5f-0b6a-4072-ac5a-4297acdeff15, 658c38e3-e30b-42ba-abf0-8025fef69713]",0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
for i, row in retrieval_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n============Error #{i+1}=============\n\n")
    print(f"Query:\n{row.query}\n")
    print(f"- Expected contexts:\n")
    expected_nodes = docstore.get_nodes(row.expected_ids)
    for node in expected_nodes:
        print(node.get_content(metadata_mode=MetadataMode.LLM))

    print(f"\n- Retrieved contexts:\n")
    retrieved_nodes = docstore.get_nodes(row.retrieved_ids)
    for i, node in enumerate(retrieved_nodes[:10]):
        print(f"+ Context #{i+1}:\n\n{node.get_content(metadata_mode=MetadataMode.LLM)}\n")



============Error #1=============


Query:
What are some restaurants that offer excellent customer service and a good dining experience?

- Expected contexts:

review_stars: 3
biz_name: Bier Brewery and Tap Room
biz_address: 5133 E 65th St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Food', 'Beer', 'Wine & Spirits', 'Breweries']

I only gave them 3stars this time but I look forward to improving their score after future visits.

- Retrieved contexts:



ValueError: doc_id 909e5e89-e168-4427-b745-291de49a670a not found.

In [50]:
def check_retrieval_results(query):
    retrieval_results = retriever.retrieve(query)
    for node in retrieval_results:
        display_source_node(node, source_length=1000)
        print(node.get_content(metadata_mode=MetadataMode.EMBED))
        print("\n")
    return retrieval_results

query = "What are some good grocery stores in Nashville that offer curated items and unique local products?"
check_retrieval_results(query)

Generated queries:
grocery stores nashville curated items local products
I love shopping at the local grocery stores in Nashville! They have such a great selection of curated items and unique local products. I always find something new and exciting to try.


**Node ID:** 7d27d2e6-a3db-4a84-9acb-b3803bb682ae<br>**Similarity:** 0.06533538146441371<br>**Text:** Some unique flavors too. Not much else to say but an amazing ice cream store that others should try to emulate.<br>

review_stars: 5
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food']

Some unique flavors too. Not much else to say but an amazing ice cream store that others should try to emulate.




**Node ID:** a8248e27-4017-4258-80f3-788eb11ed4f8<br>**Similarity:** 0.0630272952853598<br>**Text:** Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.<br>

review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.




...

**Node ID:** 4c65096c-25ff-4f75-9850-30983a2762f4<br>**Similarity:** 0.03278688524590164<br>**Text:** This place was amazing! I only tried their ice cream I really wanted to try the hot chocolate but didn't get a chance :(. Well the ice cream was great! The first night I came here I got a brownie sundae all for myself, brownie on the bottom and three scoops of ice cream topped with whipped cream and nuts. I finished it all happily! I chose pumpkin cake flavor which was delicious and smooth. I also got pralines and cream that was better than other praline ice creams. The Tennessee fudge was vanilla with fudge swirled it was good not too special. 

The next time I came I got nutter butter ice cream that was really good had cookie pieces in it! I got the hot fudge topping over it and that was a great combination. 

The service was great the workers were really nice and patient with me haha indecisive! They would joke around with me and let me sample all I wanted.<br>

review_stars: 5
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food']

This place was amazing! I only tried their ice cream I really wanted to try the hot chocolate but didn't get a chance :(. Well the ice cream was great! The first night I came here I got a brownie sundae all for myself, brownie on the bottom and three scoops of ice cream topped with whipped cream and nuts. I finished it all happily! I chose pumpkin cake flavor which was delicious and smooth. I also got pralines and cream that was better than other praline ice creams. The Tennessee fudge was vanilla with fudge swirled it was good not too special. 

The next time I came I got nutter butter ice cream that was really good had cookie pieces in it! I got the hot fudge topping over it and that was a great combination. 

The service was great the workers were really nice and patient with me hah

**Node ID:** 3514a9a8-b4fe-4194-b053-e0ab21719e2e<br>**Similarity:** 0.03200204813108039<br>**Text:** Also no judgement for eating all the brownie sundae alone :) lol<br>

review_stars: 5
biz_name: Mike's Ice Cream
biz_address: 129 2nd Ave N
biz_city: Nashville
biz_state: TN
biz_categories: ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food']

Also no judgement for eating all the brownie sundae alone :) lol




[NodeWithScore(node=TextNode(id_='7d27d2e6-a3db-4a84-9acb-b3803bb682ae', embedding=None, metadata={'review_id': '6odPSjoPWOyeDQpgJbdGaA', 'user_id': 'jLr7vcDg43TqS2OK4ROvPQ', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-03-30T23:22:54', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False,

## Response

In [62]:
from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_index.core.response_synthesizers.type import ResponseMode

from src.features.append_reference.custom_query_engine import ManualAppendReferenceQueryEngine
from src.features.synthesize_recommendation.custom_tree_summarize import CUSTOM_TREE_SUMMARIZE_PROMPT_SEL

In [59]:
node_postprocessors = []

if cfg.retrieval_cfg.retrieval_simiarity_cutoff is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=cfg.retrieval_simiarity_cutoff))

reranker = FlagEmbeddingReranker(model=cfg.retrieval_cfg.rerank_model_name, top_n=cfg.retrieval_cfg.rerank_top_k, use_fp16=True)
node_postprocessors.append(reranker)

response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.TREE_SUMMARIZE,
    summary_template=CUSTOM_TREE_SUMMARIZE_PROMPT_SEL
)
query_engine = ManualAppendReferenceQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

## Test Query Engine

In [60]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [63]:
query = "What are some places to enjoy cold brew coffee"
response = query_engine.query(query)
display_response(response, source_length=1000, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
cold brew coffee
I love trying new cold brew coffee places. My favorite is [Place Name] because they have such a unique flavor and the atmosphere is great.


**`Final Response:`** ## Cold Brew Coffee Recommendations:

Based on the provided information, here are some recommendations for places to enjoy cold brew coffee, ranked by relevance:

**1. Helena Avenue Bakery (Santa Barbara, CA):** 
* **Strengths:**  Explicitly mentions "cold brew coffee was good" in the review. 
* **Relevance:**  High - directly mentions cold brew coffee and is a bakery/cafe setting.
* **Comparison:**  While the review doesn't elaborate on the quality of the cold brew, it's a strong contender due to the direct mention.

**2. Wawa (Philadelphia, PA):**
* **Strengths:**  Known for its convenience store offerings, including coffee and tea.
* **Relevance:**  Moderate -  While not explicitly mentioning cold brew, Wawa is a known chain for coffee options.
* **Comparison:**  Less relevant than Helena Avenue Bakery due to the lack of specific mention of cold brew.

**3. Mike's Ice Cream (Nashville, TN):**
* **Strengths:**  Offers coffee and tea alongside ice cream.
* **Relevance:**  Low -  While offering coffee, the focus is on ice cream. No mention of cold brew specifically.
* **Comparison:**  Least relevant compared to the other options, as the focus is on ice cream.

**4. Roast Coffeehouse and Wine Bar (Edmonton, AB):**
* **Strengths:**  Primarily a coffeehouse, but the review focuses on negative aspects.
* **Relevance:**  Low -  While a coffeehouse, the review highlights issues with the music and lack of tea options, making it less appealing for a coffee experience.
* **Comparison:**  Not recommended due to the negative review and lack of focus on cold brew.

**5. Bar One (Philadelphia, PA):**
* **Strengths:**  A cocktail bar with a focus on brunch.
* **Relevance:**  Very Low -  No mention of coffee or cold brew.
* **Comparison:**  Not relevant to the user's query.

**6. Romano's Macaroni Grill (Reno, NV):**
* **Strengths:**  An Italian restaurant.
* **Relevance:**  Very Low -  No mention of coffee or cold brew.
* **Comparison:**  Not relevant to the user's query.

**Recommendation:**

Based on the provided information, **Helena Avenue Bakery** is the most relevant option for enjoying cold brew coffee. While Wawa is a possibility, the lack of specific mention of cold brew makes it less appealing. 



#### Reference Paragraphs
**Helena Avenue Bakery**

>Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good.  (Review ID: <REVIEW_ID>Q3fPo_x6xKxafAzy1hFITg</REVIEW_ID>)

>What a great addition to the Funk Zone!   (Review ID: <REVIEW_ID>L0jv8c2FbpWSlfNC6bbUEA</REVIEW_ID>)

>Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant. (Review ID: <REVIEW_ID>L0jv8c2FbpWSlfNC6bbUEA</REVIEW_ID>)

**Wawa**

>Wawa is like a beacon in a cold, dark world.  I grew up around Sheetz locations, but I have to admit that I prefer Wawa.  The sandwiches are really good, and they have the typical snacks/drinks you would expect from a convenience store.   (Review ID: <REVIEW_ID>8x4WVgu2t2MD48_ftQXrLw</REVIEW_ID>)

**Roast Coffeehouse and Wine Bar**

>So much to like about this place--the beautiful Mercer Building location, the spaciousness, the lovely, friendly baristas. But there are a couple of important cons for me. Biggest being, the music is constant and too loud, ruining everything you go to a coffee shop to do: read, work, reflect, or enjoy a good conversation. When I asked for it to be turned down, they told me they can't--it just plays at one level all the time, and the speakers  are everywhere so you can't get away from it. It sucks.

 (Review ID: <REVIEW_ID>uwPhE21CZLlp1kkSMPvvYw</REVIEW_ID>)

>Secondly, if you're a tea drinker, you're out of luck. They don't serve the tea in pots, but huge, sloppy individual filters--it makes your beverage totally ungainly, hard to drink and is really off putting.

Pros: gluten free goodies and yummy Italian sodas. I'd love to spend more time here, but the music usually drives me out after 10 minutes. (Review ID: <REVIEW_ID>uwPhE21CZLlp1kkSMPvvYw</REVIEW_ID>)

**Mike's Ice Cream**

>Also no judgement for eating all the brownie sundae alone :) lol (Review ID: <REVIEW_ID>zj-LN7a09KdkTSWIi8rMkw</REVIEW_ID>)

>Some unique flavors too. Not much else to say but an amazing ice cream store that others should try to emulate. (Review ID: <REVIEW_ID>6odPSjoPWOyeDQpgJbdGaA</REVIEW_ID>)

**Bar One**

>I love how creative everything was.
I CANT STOP DAYDREAMING ABOUT THE FOOD. Hands down, best brunch experience of my life. I don't even like french toast usually and theres was by far the most delicious food thats ever entered my mouth. I am literally drooling while writing this.
The chicken and waffles were unreal and standout A++++. 
I love the "flinstones pushup-pop" drink so creative and clever.
Even the deviled eggs were the best I've personally ever had. They do everything right even when it comes to the music. It was super fun and such a great experience! Ignore any review thats not 4 stars because they must be delusional. (Review ID: <REVIEW_ID>z_fgvINjKZCw5RgByaTxxw</REVIEW_ID>)

**Romano's Macaroni Grill**

>Staff are as friendly as it gets. (Review ID: <REVIEW_ID>mMwnX1vc3tQUeDNS2wiKFw</REVIEW_ID>)

---

**`Source Node 1/10`**

**Node ID:** 8c4df29f-641e-46c3-a9d8-8bee00d8c655<br>**Similarity:** -1.6597599983215332<br>**Text:** Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good.<br>**Metadata:** {'review_id': 'Q3fPo_x6xKxafAzy1hFITg', 'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-07-17T00:20:41', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'True', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries'], 'biz_hours': {'Friday': '8:0-14:0', 'Monday': '0:0-0:0', 'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0', 'Thursday': '8:0-14:0', 'Tuesday': '8:0-14:0', 'Wednesday': '8:0-14:0'}}<br>

...

**Node ID:** 553a4a2e-22ea-4b5b-a8d0-82a6b09079e3<br>**Similarity:** -8.847516059875488<br>**Text:** I love how creative everything was.
I CANT STOP DAYDREAMING ABOUT THE FOOD. Hands down, best brunch experience of my life. I don't even like french toast usually and theres was by far the most delicious food thats ever entered my mouth. I am literally drooling while writing this.
The chicken and waffles were unreal and standout A++++. 
I love the "flinstones pushup-pop" drink so creative and clever.
Even the deviled eggs were the best I've personally ever had. They do everything right even when it comes to the music. It was super fun and such a great experience! Ignore any review thats not 4 stars because they must be delusional.<br>**Metadata:** {'review_id': 'z_fgvINjKZCw5RgByaTxxw', 'user_id': 'dldfg-X_QbBkhR2DOsQFWg', 'business_id': 'QdN72BWoyFypdGJhhI5r7g', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-11-10T16:52:33', 'biz_name': 'Bar One', 'biz_address': '767 S 9th St', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19147', 'biz_latitude': 39.9398245705, 'biz_longitude': -75.1574465632, 'biz_stars': 4.0, 'biz_review_count': 65, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'full_bar'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': None, 'divey': False, 'intimate': False, 'trendy': None, 'upscale': False, 'classy': None, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': None, 'street': True, 'validated': None, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'False', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': True, 'breakfast': False}", 'HappyHour': 'True', 'HasTV': 'True', 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': None, 'WiFi': "u'free'"}, 'biz_categories': ['Cocktail Bars', 'Bars', 'Italian', 'Nightlife', 'Restaurants'], 'biz_hours': {'Friday': '12:0-2:0', 'Monday': '16:0-0:0', 'Saturday': '11:0-2:0', 'Sunday': '11:0-0:0', 'Thursday': '12:0-0:0', 'Tuesday': '16:0-0:0', 'Wednesday': '16:0-0:0'}}<br>

---

**`Source Node 10/10`**

**Node ID:** 716413ae-6eb9-4ff6-8737-959217e4ccbc<br>**Similarity:** -10.137247085571289<br>**Text:** Staff are as friendly as it gets.<br>**Metadata:** {'review_id': 'mMwnX1vc3tQUeDNS2wiKFw', 'user_id': 'f10WH1fXhy-68r4AEEhAWA', 'business_id': '9OG5YkX1g2GReZM0AskizA', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-01-30T03:16:46', 'biz_name': "Romano's Macaroni Grill", 'biz_address': '5505 S Virginia St', 'biz_city': 'Reno', 'biz_state': 'NV', 'biz_postal_code': '89502', 'biz_latitude': 39.4761165, 'biz_longitude': -119.7893392, 'biz_stars': 2.5, 'biz_review_count': 339, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "'full_bar'", 'Ambience': "{u'divey': False, u'hipster': False, u'casual': True, u'touristy': False, u'trendy': False, u'intimate': False, u'romantic': None, u'classy': None, u'upscale': False}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False}", 'HappyHour': 'True', 'HasTV': 'True', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'True', 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "u'free'"}, 'biz_categories': ['Restaurants', 'Italian'], 'biz_hours': {'Friday': '11:0-21:0', 'Monday': '11:0-22:0', 'Saturday': '12:0-21:0', 'Sunday': '11:0-21:0', 'Thursday': '11:0-20:0', 'Tuesday': '11:0-22:0', 'Wednesday': '10:0-22:0'}}<br>

{'8c4df29f-641e-46c3-a9d8-8bee00d8c655': {'review_id': 'Q3fPo_x6xKxafAzy1hFITg',
  'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ',
  'business_id': 'IDtLPgUrqorrpqSLdfMhZQ',
  'review_stars': 4,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'date': '2018-07-17T00:20:41',
  'biz_name': 'Helena Avenue Bakery',
  'biz_address': '131 Anacapa St, Ste C',
  'biz_city': 'Santa Barbara',
  'biz_state': 'CA',
  'biz_postal_code': '93101',
  'biz_latitude': 34.4144445,
  'biz_longitude': -119.6906718,
  'biz_stars': 4.0,
  'biz_review_count': 389,
  'biz_is_open': 1,
  'biz_attributes': {'AcceptsInsurance': None,
   'Alcohol': "u'none'",
   'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}",
   'BYOB': None,
   'BestNights': None,
   'BikeParking': 'True',
   'BusinessAcceptsBitcoin': None,
   'BusinessAcceptsCreditCards': 'True',
   'BusinessParking': "{'garage': False, 'street': True, 'v

## Response Evaluation

In [34]:
from src.run.eval.response import ResponseEvaluator

In [35]:
response_evaluator = ResponseEvaluator()

### Synthetic

In [38]:
response_eval_documents, response_synthetic_eval_dataset = response_evaluator.generate_synthetic_dataset(cfg, documents)

2024-09-21 17:31:39.802 | INFO     | src.run.eval.response:generate_synthetic_dataset:39 - Sampling 10 documents for response evaluations...
2024-09-21 17:31:39.803 | INFO     | src.run.eval.response:generate_synthetic_dataset:58 - Creating new response eval dataset at data/025_modify_prompt_with_tree_summarize/response_synthetic_eval_dataset.json...
2024-09-21 17:31:39.803 | INFO     | src.run.eval.response:generate_synthetic_dataset:61 - Creating synthetic response eval dataset...


Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|                              | 0/1 [00:00<?, ?it/s]


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).